# `DATA EXPLORATION`

## **TOPIC: FILMS ANALYSIS**

`Group ID`: 17

`Group Member`:
- 22127404_Tạ Minh Thư
- 22127359_Chu Thúy Quỳnh
- 22127302_Nguyễn Đăng Nhân

### **IMPORT**

In [29]:
import pandas as pd
import os

### Read data

In [2]:
file = open('films_data.csv', 'r', encoding='utf-8-sig')

data = {}
first_line = file.readline().strip().split('\t')
for val in first_line:
    data[val] = []

for line in file:
    line_vals = line.strip().split('\t')   
    for i in range(len(line_vals)):
        data[first_line[i]].append(line_vals[i])
        
data_df = pd.DataFrame(data)

### The meaning of each row
Each row represents a specific movie, detailing information about its release, performance, genre, and key contributors (director, writer, and cast).

In [3]:
n_row, n_col = data_df.shape
print('Number of columns:', n_row)

Number of columns: 1000


### The meaning of each column
- `Ranks`: The film's rank in the top lifetime grosses.
- `Titles`: The film's name.
- `Foreign %`: The percentage of the foreign grosses in the film's worldwide grosses.
- `Domestic %`: The percentage of the domestic grosses in the film's worldwide grosses.
- `Years`: The year that the film was first released.
- `Genres`: The genre(s) associated with each film.
- `Directors`: The director(s) of each film.
- `Writers`: The writer(s) credited for each film.
- `Casts`: The main cast members of each film.

In [4]:
print('Number of columns:', n_col)

Number of columns: 9


### The datatype of each column

In [5]:
data_df.dtypes

Rank          object
Title         object
Foreign %     object
Domestic %    object
Year          object
Genre         object
Director      object
Writer        object
Cast          object
dtype: object

### Preprocessing data

- Convert Percentage Columns: Convert Foreign % and Domestic % to numeric values by removing the '%' symbol and changing the data type to floats. If the value in these columns is '-', it indicates that the foreign gross accounts for 100% of the film's worldwide grosses, and the domestic gross is considered 0%.

- Standardize Year Data Type: Ensure Year is an integer for easy numerical analysis.

- Split Genres: Split the values in Genre into separate columns or lists for better analysis of each genre individually.

- Director and Writer Parsing: If needed, split multiple directors or writers into lists to analyze individual contributions.

- Cast Parsing: Similarly, parse the Cast column into individual actor names or convert to lists, which will make it easier to analyze actor appearances across movies.

In [6]:
data_df['Rank'] = data_df['Rank'].str.replace(',', '').astype(int)

data_df['Foreign %'] = data_df['Foreign %'].str.rstrip('%').astype(float)

data_df['Domestic %'] = data_df['Domestic %'].apply(
    lambda x: float(x.replace('<', '').rstrip('%')) if x != '-' else 0.0
)

data_df['Genre'] = data_df['Genre'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

data_df['Director'] = data_df['Director'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

data_df['Writer'] = data_df['Writer'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

data_df['Cast'] = data_df['Cast'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

### New datatype of each column

In [7]:
data_df.dtypes

Rank            int64
Title          object
Foreign %     float64
Domestic %    float64
Year           object
Genre          object
Director       object
Writer         object
Cast           object
dtype: object

### Check duplicated data

- `normalize_data` is created to make data comparisons easier and more consistent. This is done by cleaning up the data and making sure all values look the same regardless of formatting differences.
- The function processes each cell in the DataFrame row:
- If the cell contains a list:
    - It processes each item in the list by converting the item to lowercase.
    - Strips leading or trailing spaces.
    - Sorts the list elements using sorted() to ensure the order doesn’t matter (so ['b', 'a'] becomes ['a', 'b']).
    - Converts the cleaned list into a string format so that Pandas can handle it for comparisons.
- If the cell does not contain a list:
    - It simply converts the value to lowercase.
    - Strips any extra spaces.
- This cleaned and consistent version of the row is then returned.
- After that, we wil applies `normalize_data` function to each row of the DataFrame.
- The line `num_duplicated_rows = normalized_df.duplicated().sum()` checks for duplicates in the normalized DataFrame:
- `normalized_df.duplicated()` returns a Series (a single column of data) where each row is marked as True if it is a duplicate of a previous row.
- `.sum()` counts how many True values are in this Series, giving the total number of duplicated rows.
- If duplicates are found, it will create a new DataFrame containing all rows that are considered duplicates:
    - `keep=False` marks all rows as True if they are duplicates, including the first occurrence, so that all duplicate rows can be seen.
- The code then prints the duplicated rows for examination.
- If no duplicates are found, it simply prints a message saying, "No duplicated data found."

In [12]:
df = pd.DataFrame(data)

def normalize_data(row):
    return row.apply(
        lambda x: str(sorted([str(v).strip().lower() for v in x])) if isinstance(x, list) 
        else str(x).strip().lower()
    )

normalized_df = df.apply(normalize_data, axis=1)

num_duplicated_rows = normalized_df.duplicated().sum()
print(f"The raw data has {num_duplicated_rows} duplicated rows.")

if num_duplicated_rows > 0:
    duplicates = df[normalized_df.duplicated(keep=False)]
    print("Duplicated rows:")
    print(duplicates)
else:
    print("No duplicated data found.")

The raw data has 0 duplicated rows.
No duplicated data found.


## Check missing data

- `df.isnull()` creates a DataFrame of the same size as df, with True where values are NaN (missing) and False otherwise.
- `df.isnull().sum()` counts the number of True values (i.e., missing values) for each column.
- Dividing `df.isnull().sum()` by `len(df)` calculates the proportion of missing values for each column.
- Multiplying by 100 converts this proportion to a percentage, resulting in missing_ratio_results, which is a Pandas Series showing the percentage of missing data per column.

In [9]:
df = pd.DataFrame(data)
def check_missing_data(data):

    
    missing_ratio_results = (df.isnull().sum() / len(df)) * 100

    print("\nMissing ratio per column:")
    for col, ratio in missing_ratio_results.items():
        print(f"Column '{col}': {ratio:.2f}% missing")

    return missing_ratio_results.to_dict()

missing_ratio_results = check_missing_data(data)


Missing ratio per column:
Column 'Rank': 0.00% missing
Column 'Title': 0.00% missing
Column 'Foreign %': 0.00% missing
Column 'Domestic %': 0.00% missing
Column 'Year': 0.00% missing
Column 'Genre': 0.00% missing
Column 'Director': 0.00% missing
Column 'Writer': 0.00% missing
Column 'Cast': 0.00% missing


- Because of the missing ratio of all columns is 0.00% and there's no duplicated data, it means there is no missing data. The dataset is ready for analysis without the need for imputation or data cleaning related to missing values or duplicated data.

### **DATA DISTRIBUTION**

#### **For numeric columns**

In [10]:
numeric_cols = data_df.select_dtypes(include='number')

def lower_quartile(col):
    return col.quantile(0.25)

def upper_quartile(col):
    return col.quantile(0.75)

def median(col):
  return col.median()
min_vals = numeric_cols.min()
max_vals = numeric_cols.max()
lower_quartile_vals = numeric_cols.apply(lower_quartile)
median_vals = numeric_cols.apply(median)
upper_quartile_vals = numeric_cols.apply(upper_quartile)

num_col_info_df = pd.DataFrame({
    "min": min_vals,
    "lower_quartile": lower_quartile_vals,
    "median": median_vals,
    "upper_quartile": upper_quartile_vals,
    "max": max_vals
}).T

num_col_info_df = num_col_info_df.round(1)
num_col_info_df

,Rank,Foreign %,Domestic %
min,1.0,13.2,0.0
lower_quartile,250.8,51.1,30.6
median,500.5,60.5,39.5
upper_quartile,750.2,69.4,48.9
max,1000.0,100.0,86.8


#### **For non-numeric columns**

In [18]:
pd.set_option('display.max_colwidth', 100) # For clearly
pd.set_option('display.max_columns', None) # For clearly

In [19]:
cat_col_info = {}

for col in data_df.select_dtypes(exclude='number').columns:
    non_missing_values = data_df[col].dropna()
    non_missing_values = non_missing_values.apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)
    if data_df[col].dtype.name == 'category' or data_df[col].dtype == 'object':  # Categorical columns
        num_values = non_missing_values.nunique()

    value_counts = non_missing_values.value_counts(normalize=True) * 100
    value_ratios = value_counts.to_dict()

    cat_col_info[col] = {
        'num_values': num_values,
        'value_ratios': value_ratios
    }

cat_col_info_df = pd.DataFrame(cat_col_info).T
cat_col_info_df['value_ratios'] = cat_col_info_df['value_ratios'].apply(lambda x: {k: round(v, 1) for k, v in x.items()})
cat_col_info_df

,num_values,value_ratios
Title,988,"{'The Jungle Book': 0.2, 'The Lion King': 0.2, 'Beauty and the Beast': 0.2, 'The Little Mermaid'..."
Year,55,"{'2019': 4.9, '2017': 4.9, '2016': 4.8, '2018': 4.7, '2014': 4.5, '2013': 4.3, '2012': 4.2, '201..."
Genre,343,"{'Action,Adventure,Sci-Fi': 6.4, 'Action,Adventure,Thriller': 3.1, 'Action,Adventure,Sci-Fi,Thri..."
Director,535,"{'Steven Spielberg': 2.2, 'Tim Burton': 1.0, 'Ridley Scott': 1.0, 'Robert Zemeckis': 1.0, 'Micha..."
Writer,930,"{'M. Night Shyamalan': 0.6, 'Melissa Rosenberg,Stephenie Meyer': 0.4, 'Chris Morgan,Gary Scott T..."
Cast,980,"{'Keanu Reeves,Laurence Fishburne,Carrie-Anne Moss,Hugo Weaving': 0.3, 'Vin Diesel,Paul Walker,M..."


#### **Check constraints**

In [20]:
def checking_grosses(df: pd.DataFrame) -> bool:
    consistency_check = (df['Foreign %'] + df['Domestic %'] == 100)

    return consistency_check.all()

checking_grosses(data_df) == True

np.False_

In [26]:
sum(100 - (data_df["Foreign %"] + data_df["Domestic %"]))

-0.4999999999999716

In [27]:
data_df['Domestic %'] = 100 - data_df['Foreign %']

In [28]:
checking_grosses(data_df) == True

np.True_

In [36]:
def saveDataFrame2CSV(df: pd.DataFrame, save_path: str, sep: str = ',', encoding: str = 'utf-8') -> bool:
    try:
        df.to_csv(save_path, sep=sep, encoding=encoding, index=False)
    except:
        raise ModuleNotFoundError
        # return False
    return True

In [32]:
save_name = "cleaned_data.csv"
saveDataFrame2CSV(data_df, os.path.join("./", f"{save_name}"))

True

In [34]:
cleaned_df = pd.read_csv("cleaned_data.csv")
cleaned_df.dtypes

Rank            int64
Title          object
Foreign %     float64
Domestic %    float64
Year            int64
Genre          object
Director       object
Writer         object
Cast           object
dtype: object